<a href="https://colab.research.google.com/github/keulreobeu/deep_learning_for_the_life_sciences/blob/main/%EC%9C%A0%EC%A0%84%ED%95%99%EA%B3%BC_%EB%94%A5%EB%9F%AC%EB%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [1]:
!pip install --pre deepchem

런타임 재시작 후 다시실행


In [1]:
!pip install "keras<3" "tensorflow<2.16"

런타임 재시작 후 다시실행

In [2]:
!pip install rdkit

In [ ]:
!pip list | egrep 'deepchem|rdkit|tensorflow|keras|numpy|pandas|scikit-learn|matplotlib'


deepchem                              2.8.1.dev20250829211407
geopandas                             1.0.1
keras                                 2.15.0
keras-hub                             0.18.1
keras-nlp                             0.18.1
matplotlib                            3.10.0
matplotlib-inline                     0.1.7
matplotlib-venn                       1.1.2
numpy                                 1.26.4
pandas                                2.2.2
pandas-datareader                     0.10.0
pandas-gbq                            0.29.2
pandas-stubs                          2.2.2.240909
rdkit                                 2025.3.6
scikit-learn                          1.6.1
sklearn-pandas                        2.2.0
tensorflow                            2.15.1
tensorflow-datasets                   4.9.9
tensorflow_decision_forests           1.11.0
tensorflow-estimator                  2.15.0
tensorflow-hub                        0.16.1
tensorflow-io-gcs-filesystem         

In [3]:
import deepchem as dc
from rdkit import Chem
import numpy as np
import pandas as pd

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [ ]:
tasks, datasets, transformers = dc.molnet.load_pdbbind(
    featurizer='ECFP',      # 기본: ECFP fingerprint. (GraphConv 등도 가능)
    split='random',         # 데이터 분할 방식: random, scaffold 등
    subset='core',          # 'core', 'refined', 또는 None (전체). core가 가장 작고 빠름.
    reload=True             # 캐시 사용
)

train_dataset, valid_dataset, test_dataset = datasets

    rdkit.Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect(tuple, int)
did not match C++ signature:
    GetMorganFingerprintAsBitVect(RDKit::ROMol mol, unsigned int radius, unsigned int nBits=2048, boost::python::api::object invariants=[], boost::python::api::object fromAtoms=[], bool useChirality=False, bool useBondTypes=True, bool useFeatures=False, boost::python::api::object bitInfo=None, bool includeRedundantEnvironments=False)
    rdkit.Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect(tuple, int)
did not match C++ signature:
    GetMorganFingerprintAsBitVect(RDKit::ROMol mol, unsigned int radius, unsigned int nBits=2048, boost::python::api::object invariants=[], boost::python::api::object fromAtoms=[], bool useChirality=False, bool useBondTypes=True, bool useFeatures=False, boost::python::api::object bitInfo=None, bool includeRedundantEnvironments=False)
    rdkit.Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect(tuple, int)
did not match C++ signature:
    GetMorganFinger


### 구버전 코드
```
model = cd.models.TensorGraph(batch_size=1000)
features = layers.Feature(None, 101, 4)
labels = layers.Label(shape=(None, 1))
weights = layers.Weights(shape(None, 1))


prev = Features
for i in range(3):
     prev = layers.conv1D(
         filters=15, kernel_size=10, activation=tf.nn.relu,
         padding = 'same', in_layers=prev)
prev = layers.Dropout(dropout=0.5, in_layers=prev)


logits = layers.Dense(out_channels=1, in_layers=layers.Flatten(prev))
output = layers.Sigmoid(logits)
model.add_output(output)


loss = layers.sigmoidCrossEntropy(in_layers=[labels, logits])
weighted_loss = layers.WeightedError(in_layers=[loss, weights])
model.set_loss(weighted_loss)


train = dc.data.DiskDataset('train_dataset')
valid = dc.data.DiskDataset('valid_dataset')
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
for i in range(20):
     model.fit(train, nb_epoch=10)
     print(model.evaluate(train, [metric]))
     print(model.evaluate(valid, [metric]))
```


---

### GPT 이용해 리빌드
*   변경점
    -   split -> splitter : 공식 인자명 변경
    -   roc_auc_score -> rms, mea, r2 score : 가장 최근에 사용한 데이터인 PDBBind 사용  
    -   TensorGraph -> tf.keras Functional + dc.models.Kerasmodel : 과거 코드는 tf 1.x버전 기준으로 만들어짐 지금은 tf 2.x버전 이고 호환성, 유지보수등 이제는 사용되지 않는 모델
    -   Conv1D 1차원 데이터에서 ECFP 1차원 백터로 변환 : 들어간 데이터가 달라서



In [ ]:
import deepchem as dc
from deepchem.models.optimizers import Adam
from deepchem.models.losses import L2Loss
from tensorflow import keras
from tensorflow.keras import layers

# 1) 데이터 (회귀)
tasks, datasets, transformers = dc.molnet.load_pdbbind(
    featurizer='ECFP',
    splitter='random',
    subset='core',
    reload=True
)
train, valid, test = datasets

# 2) 모델: 선형 출력 (활성화 없음)
inputs = keras.Input(shape=(train.X.shape[1],), name="features")
x = layers.Dense(256, activation="relu")(inputs)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, name="y")(x)  # 회귀이므로 sigmoid X

keras_model = keras.Model(inputs, outputs)

model = dc.models.KerasModel(
    keras_model,
    loss=L2Loss(),                 # 회귀 손실
    optimizer=Adam(learning_rate=1e-3),
    batch_size=256,
    model_dir="pdbbind_ecfp_reg"
)

# 3) 회귀 메트릭
rmse = dc.metrics.Metric(dc.metrics.rms_score, mode="regression")
mae  = dc.metrics.Metric(dc.metrics.mae_score, mode="regression")
r2   = dc.metrics.Metric(dc.metrics.pearson_r2_score, mode="regression")

for i in range(3):
    model.fit(train, nb_epoch=3)
    print(f"[{i+1}] train:", model.evaluate(train, [rmse, mae, r2]))
    print(f"[{i+1}] valid:", model.evaluate(valid, [rmse, mae, r2]))


[1] train: {'rms_score': 1.0000000044432944, 'mae_score': 0.815701941714799, 'pearson_r2_score': nan}
[1] valid: {'rms_score': 0.9188062058699975, 'mae_score': 0.8020850866051238, 'pearson_r2_score': nan}
[2] train: {'rms_score': 1.000000061292397, 'mae_score': 0.8156961704184159, 'pearson_r2_score': nan}
[2] valid: {'rms_score': 0.9186259442659855, 'mae_score': 0.8019681419152564, 'pearson_r2_score': nan}
[3] train: {'rms_score': 1.0000000945098473, 'mae_score': 0.815706363731144, 'pearson_r2_score': nan}
[3] valid: {'rms_score': 0.918944445328111, 'mae_score': 0.8021746906205368, 'pearson_r2_score': nan}


/usr/local/lib/python3.11/dist-packages/deepchem/metrics/score_function.py:58: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  return scipy.stats.pearsonr(y, y_pred)[0]**2
/usr/local/lib/python3.11/dist-packages/deepchem/metrics/score_function.py:58: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  return scipy.stats.pearsonr(y, y_pred)[0]**2
/usr/local/lib/python3.11/dist-packages/deepchem/metrics/score_function.py:58: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  return scipy.stats.pearsonr(y, y_pred)[0]**2
/usr/local/lib/python3.11/dist-packages/deepchem/metrics/score_function.py:58: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  return scipy.stats.pearsonr(y, y_pred)[0]**2
/usr/local/lib/python3.11/dist-packages/deepchem/metrics/score_function.py:58: ConstantInputWarning: An input array is c

###교재 데이터 탐색 후 JunD 결합 여부를 따지는 모델을 만듬
-   JunD 데이터 [HCASMC Lonza 0000317155](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1503220&utm_source=chatgpt.com).bed
-   염색체 22번 시퀸스 [C22orf31 chromosome 22 open reading frame 31 ](https://www.ncbi.nlm.nih.gov/gene?Db=gene&Cmd=DetailsSearch&Term=25770).json

In [22]:
# Colab 기본: (필요시) 드라이브 마운트
# from google.colab import drive
# drive.mount('/content/drive')

import os, re, gzip, random, json
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score
from tqdm import tqdm

print("TF:", tf.__version__)


TF: 2.15.1


In [23]:
# === 입력 파일 경로 (사용자 제공) ===
BED_PATH = "/content/drive/MyDrive/data/JunD/GSM1503220_JUND_PE_q0p0001_top10k_peaks.narrowPeak.bed"
# chr22 관련 NCBI json들은 좌표/서열 추출에 직접 쓰기 어려우므로 참고용
NCBI_JSONL = "/content/drive/MyDrive/data/chromosome 22/ncbi_dataset/data/data_report.jsonl"
NCBI_CATALOG = "/content/drive/MyDrive/data/chromosome 22/ncbi_dataset/data/dataset_catalog.json"
GENE_FNA = "/content/drive/MyDrive/data/chromosome 22/ncbi_dataset/data/gene.fna"  # 보통 전체 chr22가 아님

# === chr22 레퍼런스 FASTA 경로 (있으면 지정, 없으면 2단계에서 다운로드) ===
CHR22_FASTA = "/content/chr22.fa.gz"  # 이 파일이 없으면 2단계에서 자동 다운로드
WINDOW = 101  # 101bp, 모델/전처리에서 사용
SEED = 1337
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)


In [24]:
if not os.path.exists(CHR22_FASTA):
    # UCSC hg38의 chr22만 다운로드 (압축 상태로 바로 사용 가능)
    !wget -qO /content/chr22.fa.gz http://hgdownload.soe.ucsc.edu/goldenPath/hg38/chromosomes/chr22.fa.gz
    print("Downloaded chr22 fasta to /content/chr22.fa.gz")
else:
    print("Found existing FASTA:", CHR22_FASTA)


Downloaded chr22 fasta to /content/chr22.fa.gz


In [25]:
def load_single_chrom_fasta_gz(path):
    """gz FASTA에서 chr 전체 서열을 하나의 str로 로드 (UCSC chr22.fa.gz 기준)"""
    seq_parts = []
    with gzip.open(path, "rt") as f:
        for line in f:
            if line.startswith(">"):
                continue
            seq_parts.append(line.strip())
    seq = "".join(seq_parts).upper()
    return seq

def one_hot_encode(seq):
    """A,C,G,T만 인코딩, 나머지는 N으로 취급"""
    mapping = {"A":0, "C":1, "G":2, "T":3}
    arr = np.zeros((len(seq), 4), dtype=np.float32)
    for i,ch in enumerate(seq):
        j = mapping.get(ch, None)
        if j is not None:
            arr[i, j] = 1.0
        else:
            # N 또는 기타 문자는 전부 0
            pass
    return arr

def extract_window(seq, center, window=101):
    half = window // 2
    start = center - half
    end = center + half + 1
    if start < 0 or end > len(seq):
        return None
    return seq[start:end]

def read_narrowpeak_as_df(path):
    """
    narrowPeak 형식:
    0:chrom, 1:start, 2:end, 3:name, 4:score, 5:strand(.),
    6:signalValue, 7:pValue, 8:qValue, 9:peak(정수, summit offset; 없으면 -1)
    """
    cols = ["chrom","start","end","name","score","strand",
            "signalValue","pValue","qValue","peak"]
    df = pd.read_csv(path, sep="\t", header=None, comment="#", names=cols)
    return df

def filter_chr22(df):
    # chr22 표기가 "chr22" 또는 "22" 등 다양한 경우가 있어 유연하게 필터링
    mask = df["chrom"].astype(str).str.replace("^chr","", regex=True).eq("22")
    return df[mask].copy()


In [26]:
# 4-1) chr22 서열 로드
chr22_seq = load_single_chrom_fasta_gz(CHR22_FASTA)
chr_len = len(chr22_seq)
print("chr22 length:", chr_len)

# 4-2) BED 로드 및 chr22 필터
bed = read_narrowpeak_as_df(BED_PATH)
bed22 = filter_chr22(bed)
print("chr22 peak rows:", len(bed22))

if len(bed22) == 0:
    raise ValueError("BED에서 chr22 피크가 없습니다. BED의 chrom 표기(예: chr22/22)를 확인하세요.")

# 4-3) Positive: summit 기준 101bp 추출 (peak 컬럼이 -1이면 구간 중앙 사용)
pos_seqs = []
pos_centers = []

for _,row in bed22.iterrows():
    start, end = int(row["start"]), int(row["end"])
    peak_off = int(row["peak"]) if not pd.isna(row["peak"]) else -1
    if peak_off is not None and peak_off >= 0:
        center = start + peak_off
    else:
        center = (start + end)//2

    s = extract_window(chr22_seq, center, WINDOW)
    if s is not None and "N" not in s:  # 품질을 위해 N 포함 시퀀스는 제외(선택)
        pos_seqs.append(s)
        pos_centers.append(center)

print("Positive sequences:", len(pos_seqs))

# 4-4) Negative: 피크 근처를 피해서 임의 샘플링 (동일 개수)
# 피크 구간을 확장(exclusion_margin)해서 제외 -> 음성 누수 방지
exclusion_margin = 250  # window보다 여유있게
exclusions = []
for _, row in bed22.iterrows():
    start, end = int(row["start"]), int(row["end"])
    exclusions.append((max(0, start - exclusion_margin), min(chr_len, end + exclusion_margin)))

# 빠른 조회를 위해 머지된 비겹침 interval 생성
exclusions.sort()
merged = []
for st, en in exclusions:
    if not merged or st > merged[-1][1]:
        merged.append([st, en])
    else:
        merged[-1][1] = max(merged[-1][1], en)

def is_excluded(center):
    # 이분 탐색 대신 선형으로도 충분(데이터 적정 규모 가정)
    for st, en in merged:
        if st <= center <= en:
            return True
        if center < st:
            return False
    return False

neg_seqs = []
tries = 0
target = len(pos_seqs)
while len(neg_seqs) < target and tries < target * 20:
    tries += 1
    c = random.randint(WINDOW//2, chr_len - (WINDOW//2) - 1)
    if is_excluded(c):
        continue
    s = extract_window(chr22_seq, c, WINDOW)
    if s is None or "N" in s:
        continue
    neg_seqs.append(s)

print("Negative sequences:", len(neg_seqs))
if len(neg_seqs) < target:
    print("[경고] 음성 샘플이 양성보다 적습니다. exclusion_margin을 줄이거나 tries를 늘리세요.")


chr22 length: 50818468
chr22 peak rows: 132
Positive sequences: 129
Negative sequences: 129


In [27]:
# one-hot
X_pos = np.stack([one_hot_encode(s) for s in pos_seqs], axis=0)  # (N_pos, 101, 4)
X_neg = np.stack([one_hot_encode(s) for s in neg_seqs], axis=0)  # (N_neg, 101, 4)

y_pos = np.ones((len(X_pos),), dtype=np.int64)
y_neg = np.zeros((len(X_neg),), dtype=np.int64)

X = np.concatenate([X_pos, X_neg], axis=0)
y = np.concatenate([y_pos, y_neg], axis=0)

# 섞기
idx = np.arange(len(X))
np.random.shuffle(idx)
X = X[idx]; y = y[idx]

# train/valid split (층화)
X_tr, X_va, y_tr, y_va = train_test_split(X, y, test_size=0.2, random_state=SEED, stratify=y)

print("Train:", X_tr.shape, "Valid:", X_va.shape)


Train: (206, 101, 4) Valid: (52, 101, 4)


In [28]:
def build_cnn(window=101, channels=4):
    inp = keras.Input(shape=(window, channels))
    x = layers.Conv1D(filters=15, kernel_size=10, padding="same", activation="relu")(inp)
    x = layers.Conv1D(filters=15, kernel_size=10, padding="same", activation="relu")(x)
    x = layers.Conv1D(filters=15, kernel_size=10, padding="same", activation="relu")(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation="relu")(x)
    out = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs=inp, outputs=out)
    model.compile(
        optimizer=keras.optimizers.Adam(1e-3),
        loss="binary_crossentropy",
        metrics=[keras.metrics.AUC(name="AUC"), keras.metrics.AUC(curve="PR", name="AUPRC")]
    )
    return model

model = build_cnn(WINDOW, 4)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 101, 4)]          0         
                                                                 
 conv1d (Conv1D)             (None, 101, 15)           615       
                                                                 
 conv1d_1 (Conv1D)           (None, 101, 15)           2265      
                                                                 
 conv1d_2 (Conv1D)           (None, 101, 15)           2265      
                                                                 
 dropout (Dropout)           (None, 101, 15)           0         
                                                                 
 flatten (Flatten)           (None, 1515)              0         
                                                                 
 dense (Dense)               (None, 64)                97024 

In [29]:
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_AUPRC", patience=5, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_AUPRC", factor=0.5, patience=2, verbose=1)
]

history = model.fit(
    X_tr, y_tr,
    validation_data=(X_va, y_va),
    epochs=50,
    batch_size=256,
    callbacks=callbacks,
    verbose=1
)


Epoch 1/50
1/1 [==============================] - 3s 3s/step - loss: 0.6970 - AUC: 0.5013 - AUPRC: 0.4764 - val_loss: 0.7058 - val_AUC: 0.3550 - val_AUPRC: 0.4182 - lr: 0.0010
Epoch 2/50
1/1 [==============================] - 0s 121ms/step - loss: 0.6805 - AUC: 0.6257 - AUPRC: 0.6118 - val_loss: 0.7046 - val_AUC: 0.3565 - val_AUPRC: 0.4262 - lr: 0.0010
Epoch 3/50
1/1 [==============================] - ETA: 0s - loss: 0.6790 - AUC: 0.6372 - AUPRC: 0.6108
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
1/1 [==============================] - 0s 124ms/step - loss: 0.6790 - AUC: 0.6372 - AUPRC: 0.6108 - val_loss: 0.7041 - val_AUC: 0.3691 - val_AUPRC: 0.4323 - lr: 0.0010
Epoch 4/50
1/1 [==============================] - 0s 103ms/step - loss: 0.6710 - AUC: 0.6822 - AUPRC: 0.6955 - val_loss: 0.7038 - val_AUC: 0.3787 - val_AUPRC: 0.4524 - lr: 5.0000e-04
Epoch 5/50
1/1 [==============================] - ETA: 0s - loss: 0.6648 - AUC: 0.7318 - AUPRC: 0.7169
Epoch 5: Red

In [30]:
# ROC-AUC / PR-AUC
proba_va = model.predict(X_va, verbose=0).ravel()
roc = roc_auc_score(y_va, proba_va)
pr  = average_precision_score(y_va, proba_va)
print(f"Valid ROC-AUC: {roc:.4f} | PR-AUC: {pr:.4f}")

# 임계값 0.5 기준 F1 등 간단 확인 (선택)
from sklearn.metrics import f1_score, classification_report
pred_va = (proba_va >= 0.5).astype(int)
print("F1:", f1_score(y_va, pred_va))
print(classification_report(y_va, pred_va, digits=3))


Valid ROC-AUC: 0.3506 | PR-AUC: 0.4330
F1: 0.3673469387755102
              precision    recall  f1-score   support

           0      0.414     0.462     0.436        26
           1      0.391     0.346     0.367        26

    accuracy                          0.404        52
   macro avg      0.403     0.404     0.402        52
weighted avg      0.403     0.404     0.402        52



GPT로 만든 test모델은 0.5보다 낮음 -> 모델학습이 재대로 되지 않음

In [32]:
import pandas as pd
import json

# === 1. JunD narrowPeak ===
BED_PATH = "/content/drive/MyDrive/data/JunD/GSM1503220_JUND_PE_q0p0001_top10k_peaks.narrowPeak.bed"
cols = ["chrom","start","end","name","score","strand","signalValue","pValue","qValue","peak"]
jund_df = pd.read_csv(BED_PATH, sep="\t", header=None, names=cols, comment="#")
print("📌 JunD narrowPeak 미리보기")
display(jund_df.head())

# === 2. NCBI data_report.jsonl (JSON Lines) ===
JSONL_PATH = "/content/drive/MyDrive/data/chromosome 22/ncbi_dataset/data/data_report.jsonl"
records = []
with open(JSONL_PATH, "r") as f:
    for line in f:
        records.append(json.loads(line))
data_report_df = pd.json_normalize(records)
print("📌 data_report.jsonl 미리보기")
display(data_report_df.head())

# === 3. dataset_catalog.json ===
CATALOG_PATH = "/content/drive/MyDrive/data/chromosome 22/ncbi_dataset/data/dataset_catalog.json"
with open(CATALOG_PATH) as f:
    catalog = json.load(f)
# JSON의 깊은 구조를 표 형태로 보기 쉽게 flatten
catalog_df = pd.json_normalize(catalog)
print("📌 dataset_catalog.json 미리보기")
display(catalog_df.head())

# === 4. gene.fna (FASTA) ===
FNA_PATH = "/content/drive/MyDrive/data/chromosome 22/ncbi_dataset/data/gene.fna"
# FASTA는 구조상 한 줄마다 헤더/서열이 교차하므로 테이블로 변환
headers, seqs = [], []
with open(FNA_PATH) as f:
    current_seq = []
    current_header = None
    for line in f:
        line = line.strip()
        if line.startswith(">"):
            if current_header:
                seqs.append("".join(current_seq))
            current_header = line[1:]
            headers.append(current_header)
            current_seq = []
        else:
            current_seq.append(line)
    if current_header:
        seqs.append("".join(current_seq))
gene_df = pd.DataFrame({"header": headers, "sequence": seqs})
print("📌 gene.fna 미리보기")
display(gene_df.head())


📌 JunD narrowPeak 미리보기


,chrom,start,end,name,score,strand,signalValue,pValue,qValue,peak
0,chr6,141647500,141648043,macs2/JUND_PE_q0p0001_peak_32674,5846,.,85.33320,594.12689,584.63629,305
1,chr9,125164615,125165106,macs2/JUND_PE_q0p0001_peak_38902,5775,.,84.32728,585.95526,577.57861,233
2,chr11,9587457,9587986,macs2/JUND_PE_q0p0001_peak_5693,5717,.,83.72372,579.83905,571.76343,245
3,chrX,30301517,30302156,macs2/JUND_PE_q0p0001_peak_39349,5518,.,81.30950,559.53058,551.85291,226
4,chr8,96814070,96814539,macs2/JUND_PE_q0p0001_peak_36555,5518,.,81.30950,559.53058,551.85291,253


📌 data_report.jsonl 미리보기


,annotations,chromosomes,commonName,description,ensemblGeneIds,geneGroups,geneId,orientation,proteinCount,swissProtAccessions,symbol,synonyms,taxId,taxname,transcriptCount,transcriptTypeCounts,type,nomenclatureAuthority.authority,nomenclatureAuthority.identifier
0,[{'annotationName': 'GCF_000001405.40-RS_2025_...,[22],human,chromosome 22 open reading frame 31,[ENSG00000100249],"[{'id': '25770', 'method': 'NCBI Ortholog'}]",25770,minus,6,[O95567],C22orf31,"[HS747E2A, bK747E2.1]",9606,Homo sapiens,6,"[{'count': 2, 'type': 'PROTEIN_CODING'}, {'cou...",PROTEIN_CODING,HGNC,HGNC:26931


📌 dataset_catalog.json 미리보기


,apiVersion,genes.files
0,V2,"[{'filePath': 'gene.fna', 'fileType': 'GENOMIC..."


📌 gene.fna 미리보기


,header,sequence
0,NC_000022.11:c29073943-29058672 C22orf31 [orga...,GATGTCGCTGGAGGTCTCGGAGAGGGGCGTATAGCCTCTTGTAGCG...
1,NC_060946.1:c29535171-29519948 C22orf31 [organ...,GATGTCGCTGGAGGTCTCGGAGAGGGGCGTATAGCCTCTTGTAGCG...


##### EDA중 c22orf31이 22번 염색체의 전체가 아닌것으로 확인
##### 교재를 빠르게 확인 후 Deepchem 튜토리얼에서 나온 내용을 배우기로 학습방향 전환

